In [1]:
import pymysql
import pandas as pd
import matplotlib.pyplot as plt
import decimal
import statsmodels.api as sm
import pylab
from scipy import stats

In [3]:
db = pymysql.connect(host='localhost',
                     user='root',
                     password='7788520q',
                     database='industry',
                     charset='utf8')
cursor = db.cursor()
cursor.execute("SELECT VERSION()")
data = cursor.fetchone()
print ("数据库连接成功！")

数据库连接成功！


In [4]:
sql="select * from origin_chip"
cursor.execute(sql)
origin_chip = cursor.fetchall()
origin_chip_pd=pd.DataFrame(list(origin_chip))
origin_chip_columns_names=['chain_name','udi','node1','node2','node3','node4','node5','node6','node7','node8','node9','node10','create_dt','create_person','update_dt','update_person']
origin_chip_pd.columns=origin_chip_columns_names

In [49]:
origin_chip_pd=origin_chip_pd[origin_chip_pd['chain_name']!=""]

In [50]:
origin_chip_pd.tail()

,chain_name,udi,node1,node2,node3,node4,node5,node6,node7,node8,node9,node10,create_dt,create_person,update_dt,update_person
201,{集成电路}产业链,下游,{集成电路}产品应用,IT产品,电脑存储与外围设备,None,None,None,None,None,None,None,None,None,None,None
202,{集成电路}产业链,下游,{集成电路}产品应用,IT产品,办公设备,None,None,None,None,None,None,None,None,None,None,None
203,{集成电路}产业链,下游,{集成电路}产品应用,IT产品,电脑cpu,None,None,None,None,None,None,None,None,None,None,None
204,{集成电路}产业链,下游,{集成电路}产品应用,家用电器,None,None,None,None,None,None,None,None,None,None,None,None
205,{集成电路}产业链,下游,{集成电路}产品应用,智能电网,None,None,None,None,None,None,None,None,None,None,None,None


In [107]:
all_list=[]
for i in range(origin_chip_pd.shape[1]-6):
    for j in range(origin_chip_pd.shape[0]):
        # print(i,j)
        h=tuple(str(i))+tuple(origin_chip_pd.iloc[j,i:i+2])
        # print(h)
        all_list.append(h)
k=set(all_list)
print(k)  

{('5', '通信SOC', '有线通信芯片'), ('6', 'DCU', None), ('4', '光刻机', '电子束光刻机'), ('6', '射频滤波器', 'FBAR滤波器'), ('4', '仿真工具', 'Ansoft\xa0HFSS'), ('7', 'Flash', 'NAND Flash'), ('4', '微处理器设计', None), ('5', '芯片测试机', None), ('3', '封装测试设备', '封装设备'), ('4', '逻辑设计工具', 'Synopsys\xa0Design\xa0Compiler'), ('5', '逻辑电路', '基带芯片'), ('1', '下游', '{集成电路}产品应用'), ('6', '射频开关', None), ('4', None, None), ('5', '航天器', None), ('3', '封装材料', '基板'), ('5', '高压喷洗机（HPP）', None), ('6', '打印机主控芯片', None), ('4', '硅', '硅粉'), ('6', '接口芯片', '串行接口芯片'), ('2', '{集成电路}产品应用', '信息通信'), ('4', '镀膜设备', '物理气相沉积机（PVD）'), ('4', '集成电路封装', '先进封装'), ('4', '磷化铟', None), ('4', '集成电路封装', '传统封装'), ('8', '荧光灯驱动芯片', None), ('5', 'Mentor\xa0Graphics', None), ('4', '离子注入机', '低能量离子注入机'), ('5', '信号链芯片', '比较器'), ('7', '组合逻辑电路', '数字运算电路'), ('5', '贴片机', None), ('4', '电路设计工具', 'Cadence'), ('7', '并行接口芯片', None), ('7', None, None), ('5', '分选机', None), ('6', '易失性存储芯片', 'DRAM'), ('3', '集成电路封测', '集成电路测试'), ('5', '单晶硅', '晶圆'), ('7', '背光驱动芯片', 'LED背光驱动芯片'), ('3', '设计工具ED

In [108]:
node_list=[]
for t in range(len(k)):
    if all(list(k)[t]) is True:
        node_list.append(list(k)[t])
print(node_list)

[('5', '通信SOC', '有线通信芯片'), ('4', '光刻机', '电子束光刻机'), ('6', '射频滤波器', 'FBAR滤波器'), ('4', '仿真工具', 'Ansoft\xa0HFSS'), ('7', 'Flash', 'NAND Flash'), ('3', '封装测试设备', '封装设备'), ('4', '逻辑设计工具', 'Synopsys\xa0Design\xa0Compiler'), ('5', '逻辑电路', '基带芯片'), ('1', '下游', '{集成电路}产品应用'), ('3', '封装材料', '基板'), ('4', '硅', '硅粉'), ('6', '接口芯片', '串行接口芯片'), ('2', '{集成电路}产品应用', '信息通信'), ('4', '镀膜设备', '物理气相沉积机（PVD）'), ('4', '集成电路封装', '先进封装'), ('4', '集成电路封装', '传统封装'), ('4', '离子注入机', '低能量离子注入机'), ('5', '信号链芯片', '比较器'), ('7', '组合逻辑电路', '数字运算电路'), ('4', '电路设计工具', 'Cadence'), ('6', '易失性存储芯片', 'DRAM'), ('3', '集成电路封测', '集成电路测试'), ('5', '单晶硅', '晶圆'), ('7', '背光驱动芯片', 'LED背光驱动芯片'), ('3', '设计工具EDA', '仿真工具'), ('5', '信号链芯片', '放大器'), ('5', '传统封装', 'PGA封装'), ('7', '逻辑门电路', '或门电路'), ('2', '{集成电路}技术支持', '其他芯片设计'), ('4', '模拟集成电路', '射频器件'), ('5', '先进封装', 'Embedded Die Packaging嵌入式封装'), ('4', '镀膜设备', '化学气相沉积机（CVD）'), ('4', '封装设备', '贴片机'), ('2', '{集成电路}原材料', '封装材料'), ('5', '干法蚀刻机', '反应离子束蚀刻机（RIE）'), ('6', '驱动芯片', '音频驱动芯片'), ('6', '谐振器',

In [118]:
node_chip_pd=pd.DataFrame(node_list,columns=['node_rank','father_node_name','node_name'])


In [119]:
node_chip_pd.sort_values('node_rank',inplace=True) 

In [120]:
node_chip_pd

,node_rank,father_node_name,node_name
47,0,{集成电路}产业链,下游
101,0,{集成电路}产业链,中游
38,0,{集成电路}产业链,上游
262,1,上游,{集成电路}生产设备
67,1,上游,{集成电路}技术支持
...,...,...,...
209,7,逻辑门电路,非门电路
198,7,背光驱动芯片,LCD背光驱动芯片
122,7,ROM,EPROM
268,7,照明驱动芯片,LED照明驱动芯片
